# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
csv_path = "../output_data/cities.csv"
vacay_df = pd.read_csv(csv_path)
vacay_df = vacay_df.drop('Unnamed: 0', axis=1)
vacay_df.head()

,ID,city,lat,lng,temp,humidity,cloudiness,wind speed,max temp
0,3366880,Hermanus,-34.42,19.23,57.00,98,83,8.99,57.00
1,6171633,Ucluelet,48.93,-125.55,45.09,89,100,1.01,48.00
2,6620339,Karratha,-20.74,116.85,85.62,71,0,9.53,85.62
3,5122534,Jamestown,42.10,-79.24,32.77,97,90,9.17,34.00
4,3835869,Santiago del Estero,-27.80,-64.26,78.42,46,65,6.71,78.42


In [3]:
# Humidity Heatmap
# Configure gmaps.
# Use the Lat and Lng as locations and Humidity as the weight.
gmaps.configure(api_key=g_key)
locations = vacay_df[["lat", "lng"]]
humidity = vacay_df['humidity'].astype(float)
fig = gmaps.figure()

In [4]:
humidity_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = .5)

fig.add_layer(humidity_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
vacay_filtered = vacay_df.loc[(vacay_df['temp']<80) & (vacay_df['temp']>70) & (vacay_df['wind speed']<10) & (vacay_df['cloudiness']==0)]

vacay_filtered

,ID,city,lat,lng,temp,humidity,cloudiness,wind speed,max temp
70,1272140,Dwarka,22.24,68.97,71.55,70,0,9.46,71.55
122,3471451,Arraial do Cabo,-22.97,-42.03,75.16,82,0,6.76,75.16
157,2186280,Nelson,-41.28,173.28,79.18,44,0,5.01,82.00
390,3860259,Cordoba,-31.41,-64.18,73.67,68,0,3.36,79.00
408,3661980,Tarauaca,-8.16,-70.77,74.12,87,0,2.10,74.12
439,3622716,Nicoya,10.15,-85.45,71.60,61,0,3.36,78.80
476,1001860,Giyani,-23.30,30.72,70.92,76,0,1.43,70.92
561,2294877,Tamale,9.40,-0.84,72.10,8,0,7.81,72.10


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = vacay_df
hotel_df['Hotel Name'] = ''
hotel_df['address'] = ''
hotel_df.head()

,ID,city,lat,lng,temp,humidity,cloudiness,wind speed,max temp,Hotel Name,address
0,3366880,Hermanus,-34.42,19.23,57.00,98,83,8.99,57.00,,
1,6171633,Ucluelet,48.93,-125.55,45.09,89,100,1.01,48.00,,
2,6620339,Karratha,-20.74,116.85,85.62,71,0,9.53,85.62,,
3,5122534,Jamestown,42.10,-79.24,32.77,97,90,9.17,34.00,,
4,3835869,Santiago del Estero,-27.80,-64.26,78.42,46,65,6.71,78.42,,


In [7]:

params = {
    "rankby": "distance",
    "type": "lodging",
    "key": g_key
    }
    
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
for index, row in hotel_df.iterrows():

    # get location from df
    target_location = (f"{row['lat']},{row['lng']}")
    target_city = row['city']

    # add location to params dict
    params['location'] = target_location
    
    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {target_city}.")
    response = requests.get(url, params=params).json()
    
    # extract results
    results = response['results']
    
                       
                       
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        hotel_df.loc[index, 'address'] = results[0]['plus_code']['compound_code']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")


Retrieving Results for Index 0: Hermanus.
Closest hotel is Hermanus Bed and Breakfast.
------------
Retrieving Results for Index 1: Ucluelet.
Closest hotel is A Snug Harbour Inn.
------------
Retrieving Results for Index 2: Karratha.
Closest hotel is ibis Styles Karratha.
------------
Retrieving Results for Index 3: Jamestown.
Closest hotel is DoubleTree by Hilton Jamestown.
------------
Retrieving Results for Index 4: Santiago del Estero.
Closest hotel is Hotel Nuevo America.
------------
Retrieving Results for Index 5: Khatanga.
Closest hotel is Gostinitsa "Zapolyar'ye".
------------
Retrieving Results for Index 6: Chumikan.
Closest hotel is Fortuna.
------------
Retrieving Results for Index 7: La Sarre.
Closest hotel is Motel Villa Mon Repos.
------------
Retrieving Results for Index 8: George Town.
Closest hotel is Silence Sea Library Hostel.
------------
Retrieving Results for Index 9: Pevek.
Closest hotel is Severnoye Zoloto.
------------
Retrieving Results for Index 10: Tasiilaq

Closest hotel is Gina's Guesthouse - Upernavik.
------------
Retrieving Results for Index 88: Vaini.
Closest hotel is Far East lodge.
------------
Retrieving Results for Index 89: Porto Velho.
Closest hotel is Fazenda Búzios- Porto Velho.
------------
Retrieving Results for Index 90: Zaprudnya.
Closest hotel is Villa Krasivaya.
------------
Retrieving Results for Index 91: Ribeira Grande.
Closest hotel is Quinta Das Buganvílias.
------------
Retrieving Results for Index 92: Yellowknife.
Closest hotel is Our Cabin Bed & Breakfast.
------------
Retrieving Results for Index 93: Kaitangata.
Closest hotel is Kaitangata Motor Camp.
------------
Retrieving Results for Index 94: Piacabucu.
Closest hotel is Portal Pousada.
------------
Retrieving Results for Index 95: Hilo.
Closest hotel is Dolphin Bay Hotel.
------------
Retrieving Results for Index 96: Charlestown.
Closest hotel is Kirinari Hostel.
Missing field/result... skipping.
------------
Retrieving Results for Index 97: Vestmannaeyjar.

Closest hotel is Amadou.
------------
Retrieving Results for Index 174: Lianzhou.
Closest hotel is Changcheng Hotel.
------------
Retrieving Results for Index 175: Vila Velha.
Closest hotel is Hotel Prainha.
------------
Retrieving Results for Index 176: Ulaangom.
Closest hotel is Baylagod hotel.
------------
Retrieving Results for Index 177: Abu Samrah.
Missing field/result... skipping.
------------
Retrieving Results for Index 178: Portland.
Closest hotel is Dossier.
------------
Retrieving Results for Index 179: Tazovsky.
Closest hotel is Hotel Tazovchanka.
------------
Retrieving Results for Index 180: Yinchuan.
Closest hotel is Yinchuan Love Space Theme Hotel.
Missing field/result... skipping.
------------
Retrieving Results for Index 181: Tomatlan.
Closest hotel is Norma franco.
------------
Retrieving Results for Index 182: Wanaka.
Closest hotel is Mantells Luxury Lodge Wanaka.
------------
Retrieving Results for Index 183: Borovoy.
Closest hotel is Ano "Al'ternativa".
---------

Closest hotel is Pangnirtung Fjordview Bed and Breakfast.
------------
Retrieving Results for Index 261: Arrondissement de Bourges.
Closest hotel is Thomas.
------------
Retrieving Results for Index 262: Hasaki.
Closest hotel is Izuya Business Hotel.
------------
Retrieving Results for Index 263: Lorengau.
Closest hotel is Seeadler Bay Hotel.
------------
Retrieving Results for Index 264: Eureka.
Closest hotel is Royal Inn Motel.
------------
Retrieving Results for Index 265: Matveyev Kurgan.
Closest hotel is Per. Sportivnyy 21.
------------
Retrieving Results for Index 266: Labuhan Deli.
Closest hotel is Warung rizkia.
------------
Retrieving Results for Index 267: Nabire.
Closest hotel is Hotel Grand Papua Nabire.
------------
Retrieving Results for Index 268: Isla Aguada.
Closest hotel is Parador Turistico Isla Aguada.
------------
Retrieving Results for Index 269: Ayr.
Closest hotel is Mercure Ayr Hotel.
------------
Retrieving Results for Index 270: Bambanglipuro.
Closest hotel is

Closest hotel is Hotel Mi Tampi.
------------
Retrieving Results for Index 347: Meadow Lake.
Closest hotel is JKL Inc. R.V Park and Storage.
------------
Retrieving Results for Index 348: Sol'-Iletsk.
Closest hotel is Gostinitsa Ordzhonikidze 35.
------------
Retrieving Results for Index 349: Pisco.
Closest hotel is Hotel Presidente.
------------
Retrieving Results for Index 350: Husavik.
Closest hotel is Spirit North Yoga & Guesthouse Húsavík.
------------
Retrieving Results for Index 351: Loudi.
Closest hotel is Highway Hotel.
------------
Retrieving Results for Index 352: Inhambane.
Closest hotel is Hotel Casa Do Capitao.
------------
Retrieving Results for Index 353: Nyahururu.
Closest hotel is Thomson Falls Resort.
------------
Retrieving Results for Index 354: Meulaboh.
Closest hotel is Hotel Tiara Meulaboh.
------------
Retrieving Results for Index 355: Rafai.
Missing field/result... skipping.
------------
Retrieving Results for Index 356: Tommot.
Closest hotel is Gostinitsa Tom

Closest hotel is Gostevoy Dom Turgoyak.
------------
Retrieving Results for Index 433: Sydney.
Closest hotel is قنصليه عراقيه الجديد.
------------
Retrieving Results for Index 434: Labuhan.
Closest hotel is bador sepret.
------------
Retrieving Results for Index 435: Laurinburg.
Closest hotel is Holiday inn express.
------------
Retrieving Results for Index 436: North Bend.
Closest hotel is City Center Motel.
------------
Retrieving Results for Index 437: Saint-Michel-des-Saints.
Closest hotel is Hotel Central.
------------
Retrieving Results for Index 438: Adrar.
Missing field/result... skipping.
------------
Retrieving Results for Index 439: Nicoya.
Closest hotel is Apartamentos Doña Gladys en Nicoya, Guanacaste, Costa Rica.
------------
Retrieving Results for Index 440: Huenfeld.
Closest hotel is Wohnmobilstellplatz Hessisches Kegelspiel.
------------
Retrieving Results for Index 441: Bac Lieu.
Closest hotel is Nhà 38/3 Trần Phú.
------------
Retrieving Results for Index 442: Pedro 

Closest hotel is Gostinitsa Vilyuy.
------------
Retrieving Results for Index 518: Raudeberg.
Closest hotel is Camping Refviksanden.
------------
Retrieving Results for Index 519: Basco.
Closest hotel is Katuvang Bed & Breakfast.
------------
Retrieving Results for Index 520: Fairbanks.
Closest hotel is Town House Motel & Apartments.
------------
Retrieving Results for Index 521: Collie.
Closest hotel is Collie Forest Motel.
------------
Retrieving Results for Index 522: Umm Lajj.
Closest hotel is Moon light Furnished Units.
Missing field/result... skipping.
------------
Retrieving Results for Index 523: Bulawayo.
Closest hotel is Charter House.
------------
Retrieving Results for Index 524: Mehar.
Closest hotel is Cafe NIZAM HOTEL.
------------
Retrieving Results for Index 525: Along.
Closest hotel is P&P Resort.
------------
Retrieving Results for Index 526: La Romana.
Closest hotel is Las Piñas 7.
------------
Retrieving Results for Index 527: Shakhtinsk.
Closest hotel is Гостинично

In [8]:
hotel_df.head(3)

,ID,city,lat,lng,temp,humidity,cloudiness,wind speed,max temp,Hotel Name,address
0,3366880,Hermanus,-34.42,19.23,57.00,98,83,8.99,57.00,Hermanus Bed and Breakfast,"H6HH+QJ Hermanus, South Africa"
1,6171633,Ucluelet,48.93,-125.55,45.09,89,100,1.01,48.00,A Snug Harbour Inn,"WFM3+98 Ucluelet, BC, Canada"
2,6620339,Karratha,-20.74,116.85,85.62,71,0,9.53,85.62,ibis Styles Karratha,"7R8W+MQ Karratha, Western Australia, Australia"


In [9]:
 # NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>Address</dt><dd>{address}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["lat", "lng"]]

In [10]:
# Add marker layer ontop of heat map
hotel_layer = gmaps.symbol_layer(locations, 
                                  fill_color='rgba(0, 150, 0, 0.4)',
                                  stroke_color='rgba(0, 0, 150, 0.4)',
                                  scale=2,
                                 info_box_content = hotel_info)
fig.add_layer(hotel_layer)
fig

# Display Map

Figure(layout=FigureLayout(height='420px'))